## Imports

In [21]:
import RPi.GPIO as GPIO
import spidev
import time
import json
import requests

## Setup Code

In [22]:
# Pin Setup
temp_pin = 3
buzzer_pin = 4
LED_R_pin = 2

# Setup GPIO
GPIO.setmode(GPIO.BCM)
GPIO.setup(buzzer_pin, GPIO.OUT)
GPIO.setup(LED_R_pin, GPIO.OUT)
buzz = GPIO.PWM(buzzer_pin, 100)

# Setup SPI
spi = spidev.SpiDev()
spi.open(0, 0)
spi.max_speed_hz = 1_000_000

### Temp Read Function

In [2]:
def temp_read(channel):
    r = spi.xfer2([1, (0x08 + channel) << 4, 0])
    adc_out = ((r[1] & 0x03) << 8) + r[2]
    voltage = adc_out * (3.3 / 1023) * 1000
    temperature = voltage / 10.0
    
    print(temperature)
    return temperature

In [30]:
spi = spidev.SpiDev()
spi.open(0, 0)
spi.max_speed_hz = 1_000_000

try:
    while True:
        temp = temp_read(temp_pin)
        print('Temperature =', round(temp, 2))
        time.sleep(1)

except KeyboardInterrupt:
    print('\nSTOP')
    pass

finally:
    spi.close()

Temperature = 21.61
Temperature = 21.61
Temperature = 21.29
Temperature = 21.61
Temperature = 21.61
Temperature = 21.61
Temperature = 21.61

STOP


### Msg Send Function

In [41]:
def send_msg():
    # User Token
    usr_token = "zoPSqzZP81dswlWQ1KS9hM9T1vszRqaOf0KvBjdzCj11WgAAAYVOZE1_"

    url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"

    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
        "Authorization": "Bearer " + usr_token
    }

    data = {
        "template_object": json.dumps({
            "object_type": "text",
            "text": "Intruder Detected!!",
            "link": {
                "web_url": "https://www.google.com",
                "mobile_web_url": "https://www.google.com"
            }
        })
     }

    # send kakao msg
    response = requests.post(url, headers=headers, data=data)
    print(response.status_code)
    if response.json().get('result_code') == 0:
        print('Message send succeeded.')
    else:
        print('Message send failed: ' + str(response.json()))

In [42]:
send_msg()

200
Message send succeeded.


### Alarm Ring Function

In [47]:
def alarm_led():
    buzz.start(1)
    GPIO.output(LED_pin, True)
    time.sleep(0.5)
    buzz.stop()
    GPIO.output(LED_pin, False)
    

In [48]:
# Pin Setup
LED_pin = 2
temp_pin = 3
buzzer_pin = 4

# Setup GPIO
GPIO.setmode(GPIO.BCM)
GPIO.setup(LED_pin, GPIO.OUT)
GPIO.setup(buzzer_pin, GPIO.OUT)
buzz = GPIO.PWM(buzzer_pin, 100)

spi = spidev.SpiDev()
spi.open(0, 0)
spi.max_speed_hz = 1_000_000

alarm_state = 0

try:
    while True:
        if alarm_state == 1:
            # ring buzzer
            alarm_led()
        if temp_read(3) > 22:
            alarm_state = 1
            # send_msg()  # send kakao msg
            
        time.sleep(1)

except KeyboardInterrupt:
    print('\nSTOP')
    pass

finally:
    spi.close()
    buzz.stop()
    GPIO.cleanup()


STOP
